In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Functions from the manuscript.
def calc_policy(Q):
    policy=np.zeros(5)
    for s in range(0,5):
        action_idx=np.argmax(Q[s,:])
        policy[s]=2*action_idx-1
        policy[0]=policy[4]=0
    return policy.astype(int)

def tau(s,a):
    if s==0 or s==4:  
        return(s)
    else:      
        l = a -1 if a[0]==0 else a;
        return(s + l)
        
def rho(s,a):
    return(s==1 and a==0)+2*(s==3 and a==1)


#Reference :
#    https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0

with tf.device('/gpu:0'):
    
    tf.reset_default_graph()
    
    # Define inputs, weights and Q values along wtih the predict values.
    inputs1 = tf.placeholder(shape=[1,5],dtype=tf.float32)
    W = tf.Variable(tf.random_uniform([5,2],0,0.01))
    Q_matrix = tf.matmul(inputs1,W)
    pred = tf.argmax(Q_matrix,1)
    
    # using mean square error loss function  and minimizing it using gradient descent optimizer 
    nextQ = tf.placeholder(shape=[1,2],dtype=tf.float32)
    loss = tf.reduce_sum(tf.square(nextQ - Q_matrix))
    grad_descent_optim = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    optimized_model = grad_descent_optim.minimize(loss) 

# Hyperparamters
y = .5
e = 0.1
episodes = 5000
targetQ = None

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(episodes):
        s = 2
        all_r = 0
        d = False
        j = 0
        while j < 100:
            j+=1
            a,all_Q = sess.run([pred, Q_matrix],feed_dict = {inputs1:np.identity(5)[s:s+1]})
            if np.random.rand() < e:
                a[0] = np.random.randint(2)
            r = rho(s, a)
            transistion_state = tau(s, a)[0]
            d = True if r>0 else False
            Q1 = sess.run(Q_matrix,feed_dict={inputs1:np.identity(5)[transistion_state:transistion_state+1]})
            maxQ1 = np.max(Q1)
            targetQ = all_Q
            targetQ[0,a[0]] = r + y*maxQ1
            _,W1 = sess.run([optimized_model, W], feed_dict = {inputs1:np.identity(5)[s:s+1],nextQ:targetQ})
            all_r += r
            s = transistion_state
            if d == True:
                if i%1000 == 0: 
                    print("episode completed {0}".format(i))
                e = 1./((i/50) + 10)
                break
    result = sess.run(W)
    policy = calc_policy(result)
    print('Policy: \n',policy.T)
    print('Q values: \n',np.around(result, decimals=1).T)


episode completed 0
episode completed 1000
episode completed 2000
episode completed 3000
episode completed 4000
Policy: 
 [ 0 -1  1  1  0]
Q values: 
 [[0.  1.  0.5 0.5 0. ]
 [0.  0.2 1.  2.  0. ]]
